In [ ]:
RESULTS_DIR = "SET ME!"

In [1]:
from dataclasses import dataclass

from code_contests_analysis import cc_dataset
from pathlib import Path

from typing import List

In [ ]:
all_problems = cc_dataset.load_dataset_into_data_structures()

problems = []
for problem in all_problems:
    # only include problems from the test set
    if problem.dataset_split != cc_dataset.DatasetSplit.TEST:
        continue

    # only include problems with valid python3 solutions
    for solution in problem.solutions:
        if solution.is_correct:
            problems.append(problem)
            break


In [ ]:
# Modifies the solutions field
_ = cc_dataset.load_test_case_runs(problems, results_dir=Path(RESULTS_DIR))

In [49]:
class TestCategoryReport:
    passing: List[cc_dataset.TestCaseRun]
    failing: List[cc_dataset.TestCaseRun]
    timeouts: List[cc_dataset.TestCaseRun]
    crashing: List[cc_dataset.TestCaseRun]

    def __init__(self, test_case_runs: List[cc_dataset.TestCaseRun]):
        self.passing = []
        self.failing = []
        self.timeouts = []
        self.crashing = []

        for run in test_case_runs:
            if run.is_correct:
                self.passing.append(run)
            elif run.timed_out:
                self.timeouts.append(run)
            elif run.crashed:
                self.crashing.append(run)
            else:
                self.failing.append(run)

    @property
    def num_passing(self):
        return len(self.passing)

    @property
    def num_failing(self):
        return len(self.failing)

    @property
    def num_timeouts(self):
        return len(self.timeouts)

    @property
    def num_crashing(self):
        return len(self.crashing)
    
    def __repr__(self):
        return (f"<TestCategoryReport(passing={self.num_passing}, "
                f"failing={self.num_failing}, timeouts={self.num_timeouts}, "
                f"crashing={self.num_crashing})>")


class SolutionTestReport:
    public_and_private_tests_report: TestCategoryReport
    generated_tests_report: TestCategoryReport

    def __init__(self, solution: cc_dataset.Solution):
        assert solution.is_correct

        public_private_runs = [
            test_case_run
            for test_case_run in solution.test_case_runs
            if test_case_run.test_case.source == cc_dataset.TestCaseSource.PUBLIC
            or test_case_run.test_case.source == cc_dataset.TestCaseSource.PRIVATE
        ]
        generated_runs = [
            test_case_run
            for test_case_run in solution.test_case_runs
            if test_case_run.test_case.source == cc_dataset.TestCaseSource.GENERATED
        ]

        assert len(public_private_runs) != 0

        self.public_and_private_tests_report = TestCategoryReport(public_private_runs)
        self.generated_tests_report = TestCategoryReport(generated_runs)

    
    def __repr__(self):
        return (f"<SolutionTestReport(public_and_private_tests_report={self.public_and_private_tests_report}, "
                f"generated_tests_report={self.generated_tests_report})>")

    def __str__(self):
        return self.__repr__()

In [50]:
reports_by_problem = []
for problem in problems:
    reports_by_problem.append(
        [
            SolutionTestReport(solution)
            for solution in problem.solutions
            if solution.is_correct
        ]
    )

In [51]:
false_negatives = []


for reports in reports_by_problem:
    for report in reports:
        public_priv_report = report.public_and_private_tests_report
        if (
            public_priv_report.num_passing > 0
            and public_priv_report.num_timeouts == 0
            and public_priv_report.num_crashing == 0
        ):
            if (
                report.generated_tests_report.num_failing > 0 or report.generated_tests_report.num_timeouts > 0 or
                report.generated_tests_report.num_crashing > 0
            ):
                false_negatives.append(report)
                break

In [ ]:
len(false_negatives)

In [ ]:
for fn in false_negatives:
    print(fn)